In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-09 15:37:20--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt'

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2024-04-09 15:37:21 (11.3 MB/s) - 'input.txt' saved [1115394/1115394]



In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [18]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [16]:
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for i, s in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
print(encode("howdy"))
print(decode(encode("howdy")))

[46, 53, 61, 42, 63]
howdy


In [19]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [20]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [21]:
# max size of chunk of text that will be fed into transformer for training
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [22]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target : {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [54]:
torch.manual_seed(21)
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

# for b in range(batch_size):
#     for t in range(block_size):
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f"when input is {context.tolist()} the target: {target}")

inputs: 
torch.Size([4, 8])
tensor([[43, 39, 56, 58, 57,  2,  0,  0],
        [ 1, 45, 53, 53, 42,  1, 58, 53],
        [44,  1, 39, 52, 53, 58, 46, 43],
        [58, 61, 47, 52, 52,  5, 42,  1]])
targets:
torch.Size([4, 8])
tensor([[39, 56, 58, 57,  2,  0,  0, 32],
        [45, 53, 53, 42,  1, 58, 53,  1],
        [ 1, 39, 52, 53, 58, 46, 43, 56],
        [61, 47, 52, 52,  5, 42,  1, 50]])
----


In [62]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensors of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else: 
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) #(B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)   
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
# Expect a loss of -ln(1/65), 65 vocab size

torch.Size([256, 65])
tensor(4.5781, grad_fn=<NllLossBackward0>)


In [63]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [68]:
batch_size = 32
for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5007448196411133


In [71]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


ndou t! y, d:
Sind he
ESTur lse nd m;
AUpord nthod G bo t we
Thin him.
Twhat hat
PUCH:
Whiere; y I s yotiouris te t
Imme imf un.
F har mas sico thook he'ZELeard athacongeawnd,
Shin as ell.

LAnus, neelloom oranto the t depa!
Ed caclie mernceshathe
Thor asue:
Fioweds
PO:
IAnd wate,
CEGHAnofeserng are


In [91]:
# Mathematical trick of self-attention
torch.manual_seed(337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)

In [100]:
# We want x[b,t] = mean_{i<=t} x[b,i]
# version 1: for loops
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        # in batch b, pluck out the channels up to time t -> (t, C)
        xprev = x[b, :t+1] 
        # in batch b at time t, fill in xbow to the mean channel value of all prev channel values
        xbow[b,t] = torch.mean(xprev, 0) 

In [90]:
a = torch.tril(torch.ones(3,3))
a = a / a.sum(1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print("a=")
print(a)
print("b=")
print(b)
print("c=")
# now c contains the weighted average at each index of itself and all previous indices
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[0., 7.],
        [8., 1.],
        [0., 7.]])
c=
tensor([[0.0000, 7.0000],
        [4.0000, 4.0000],
        [2.6667, 5.0000]])


In [103]:
# version 2: batched matrix multiplication
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B,T,T) @ (B,T,C) -----> (B,T,C)
xbow2[0], xbow[0]

(tensor([[-1.3601, -0.0852],
         [-0.3481,  0.2932],
         [ 0.0487,  0.7294],
         [-0.3293,  0.5609],
         [-0.4773,  0.6629],
         [-0.4907,  0.4857],
         [-0.4774,  0.3398],
         [-0.3369,  0.1442]]),
 tensor([[-1.3601, -0.0852],
         [-0.3481,  0.2932],
         [ 0.0487,  0.7294],
         [-0.3293,  0.5609],
         [-0.4773,  0.6629],
         [-0.4907,  0.4857],
         [-0.4774,  0.3398],
         [-0.3369,  0.1442]]))

In [105]:
# version 3: softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow3, xbow2)

# wei multiplied with our data x gives a weighted aggregate of each token's channels with all its predecessors

True

In [117]:
# version 4: self-attention
B,T,C = 4, 8, 32 # batch, time, channel
x = torch.randn(B,T,C)

# single head 
head_size = 16
key = nn.Linear(C, head_size, bias=False) # what token j contains
query = nn.Linear(C, head_size, bias=False) # what token j is looking for
value = nn.Linear(C, head_size, bias=False) # token j's embedded value
k = key(x)   # (B,T,head_size)
q = query(x) # (B,T,head_size)
wei = q @ k.transpose(-2,-1) # (B,T,T) -> affinity mapping of all tokens, contains positional and semantic info
    # a query and key is emitted by each token, multiplied with all others

    # clamp affinities from future tokens, only allowing values from preceding ones
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
v = value(x)
out = wei @ v
    # consider x, i.e. input data, as private information to this token
    

tensor([[ 2.6854, -0.8387,  1.9840,  1.0876, -2.5696, -0.4025,  0.2806,  1.5949],
        [-0.5153, -1.5526,  0.3401,  0.1211,  1.9111, -2.2460, -1.7607, -1.5664],
        [-1.0506,  0.3791, -0.1048,  0.2733, -1.5655,  1.4334, -0.0283,  1.2814],
        [-1.1091, -0.2867,  1.5406,  1.4509, -1.4658,  0.0739,  0.1352,  1.8807],
        [-0.8177, -0.5080, -2.5721, -0.9532,  1.9040, -1.6418, -1.1342, -3.4002],
        [ 1.8489, -1.7949, -2.4903, -2.0703, -0.9843,  0.5900,  1.4365,  2.1327],
        [ 0.3742, -0.7674, -1.8792, -0.3616,  1.1102,  2.5161, -1.1678,  0.8767],
        [-2.9486, -0.6349,  3.5813,  1.1695,  3.4126, -1.5975,  0.1552,  0.4181]],
       grad_fn=<SelectBackward0>)


In an input tensor X of dimension (i,j), the transfomer is being trained on (ixj) independent examples.

Attention is a communication mechanism; you can think about it as nodes in a directed graph. Each node has a vector of information and gets to aggregate information via a weighted sum from all other nodes pointing to it. 

Attention has no notion of space, just acting on a set of vectors. This is why we need to positionally encode tokens to introduce an idea of space. This is unlike a convolution where space is imoportant.

Each example across batch dimension is processed independently and never "talk" to each other.

An "encoder" block simply removes the masking/clamping of weights (wei) so all nodes can talk to each other. A "decoder" block masks the weights to decode language in a time-dependent manner.

Self-attention refers to when keys and values come from the same input data x as queries. In cross attention, the queries get produced from x but the keys and values come from some other external source.

"Scaled" attention additionally divides wei by 1/sqrt(head_size). This makes it so that when Q, K have unit variance, wei will also have unit variance, and softmax won't be too saturated. Else wei will have variance on the order of head size and softmax output will tend to one-hot vectors in the limit.

In [123]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1) * head_size**-0.5

In [124]:
k.var()

tensor(1.0333)

In [125]:
q.var()

tensor(0.9118)

In [126]:
wei.var()

tensor(0.9672)

In [310]:
# hyperparameters 
batch_size = 64 # how many independent sequences will we process in parallel
block_size = 72 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 26 # token embedding dimension
n_head = 3
n_layer = 2
dropout = 0.2
# -------------

In [311]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [312]:
class Head(nn.Module):
    """one head of self attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # a query and key is emitted by each token, multiplied with all others
        B,T,C = x.shape 
        k = self.key(x)   # (B,T,head_size) 
        q = self.query(x) # (B,T,head_size)
        # -> affinity mapping of all tokens, contains positional and semantic info
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,head_size) @ (B,head_size,T) ->  (B,T,T) 
        # masked_fill -> decoder, i.e. decoding language by clamping affinities 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)
        wei = F.softmax(wei, dim=1) # (B,T,T)
        wei = self.dropout(wei)
        
        # pool the values (weighted aggregate)
        v = self.value(x) # (B,T,head_size)
        out = wei @ v # (B,T,T) @ (B,T,head_size) -> (B,T,head_size)
        return out

class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size*num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # concatenate all head results over channel dimension
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B,T,head_size*num_heads)
        out = self.proj(out) # (B,T,n_embd)
        out = self.dropout(out)
        return out
    
class FeedForward(nn.Module):
    """a simple linear layer followed by a non-linearity"""
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(n_embd, 4*n_embd), 
                nn.ReLU(),
                nn.Linear(4*n_embd, n_embd),
                nn.Dropout(dropout),
        )
        
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """Transformer block: communication followed by computation"""
    
    def __init__(self, n_embd, n_head):
        
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_embd, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        # skip connection to improve optimization/gradient flow for deep networks
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x)) # (B,T,n_embd)
        return x

class Transformer(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) 
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensors of integers
        token_emb = self.token_embedding_table(idx) # (B,T,n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,n_embd)
        x = pos_emb + token_emb # (B,T,n_embd)
        x = self.blocks(x) # apply multi head self attention. # (B,T,n_embd)
        x = self.ln_f(x) # (B,T,n_embd) -> each token goes through this MLP independently
        logits = self.lm_head(x) # (B,T,n_embd) @ (B,n_embd,vocab_size) -> (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else: 
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)   
        return idx

In [313]:
xb, yb = get_batch('train')   
model = Transformer()
m = model.to(device)


# for name, j in model.named_parameters():
#     print(name)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
    
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']: .4f}, val loss {losses['val']: .4f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step() 


print(loss.item())
context = torch.zeros((1,1), dtype=torch.long, device=device)

step 0: train loss  4.3270, val loss  4.3284
step 500: train loss  1.6993, val loss  1.7183
step 1000: train loss  0.3925, val loss  0.4433
step 1500: train loss  0.1675, val loss  0.2061


KeyboardInterrupt: 

In [314]:
print(sum(p.numel() for p in m.parameters()), 'parameters')

59969 parameters


In [316]:
print(decode(model.generate(context, max_new_tokens=72)[0].tolist()))


LUSSEss lado ratlll llounnsndsstouun  BrBARRAEE:
o FEE faaaau'n ne oo, '
